In [6]:
import os
import sys
import tyro
import time
import random
import warnings

import numpy as np
import torch

oc_atari_dir = os.getenv("OC_ATARI_DIR")

if oc_atari_dir is not None:
    a = os.path.join(Path(__file__), oc_atari_dir)
    sys.path.insert(1, a)

sys.path.append("..")
has_agent= False

In [7]:
env_id = "ALE/Pong-v5"
obs_mode = "masked_dqn"
pth = "/data/oc_cleanrl/cleanrl/wandb/run-20241010_060604-pzk9e36i/files/DQNLv2.cleanrl_model"
architecture = "DQNLv2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

In [9]:
from ocatari.core import OCAtari
env = OCAtari(
    env_id, hud=False, render_mode="rgb_array",
        render_oc_overlay=False, obs_mode=obs_mode,
        # logger=logger, feature_func=feature_func,
        # buffer_window_size=window_size
)

obs, info = env.reset()

Using OBJ State Representation


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [10]:
if architecture == "OCT":
    from architectures.transformer import OCTransformer as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks, device).to(device)
elif architecture == "VIT":
    from architectures.transformer import VIT as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "VIT2":
    from architectures.transformer import SimpleViT2 as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "MobileVit":
    from architectures.transformer import MobileVIT as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "MobileVit2":
    from architectures.transformer import MobileViT2 as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "PPO":
    from architectures.ppo import PPODefault as Agent
    agent = Agent(env, device).to(device)
else:
    from architectures.ppo import PPO_Obj as Agent
    agent = Agent(env, device).to(device)


In [11]:
ckpt = torch.load(pth, map_location=torch.device('cpu'))
agent.load_state_dict(ckpt["model_weights"]) 

<All keys matched successfully>

In [12]:
obs = torch.from_numpy(obs).to(device)
obs = obs.unsqueeze(0)
obs.shape

torch.Size([1, 4, 84, 84])

In [13]:
import numpy as np
import matplotlib.pyplot as plt
def print_obs(obs):
    
    # Display the array as an image
    plt.imshow(obs, cmap='gray')  # Use 'gray' colormap for 2D arrays
    plt.axis('off')
    plt.show()  

In [15]:
for i in range(21):
    done = False
    crew = 0
    while not done:
        #action, _, _, _ = agent.get_action_and_value(obs)
        action = env.action_space.sample() # random moves
        obs, reward, terminated, truncated, info = env.step(action)
        
        #print_obs(obs[0])

        obs = torch.from_numpy(obs).to(device)
        obs = obs.unsqueeze(0)
        crew += reward

        

        if terminated or truncated:
            print(f"{env_id} (H): Reward is episode {i} is", crew, f"Length is episode {i} is",info["episode_frame_number"])
            #run.log({f"{opts.game}_reward": crew, f"{opts.game}_episode_length": info["episode_frame_number"]})
            observation, info = env.reset()
            done = True

env.close()

ALE/Pong-v5 (H): Reward is episode 0 is -20.0 Length is episode 0 is 4445
ALE/Pong-v5 (H): Reward is episode 1 is -21.0 Length is episode 1 is 3764
ALE/Pong-v5 (H): Reward is episode 2 is -21.0 Length is episode 2 is 3056
ALE/Pong-v5 (H): Reward is episode 3 is -20.0 Length is episode 3 is 4434
ALE/Pong-v5 (H): Reward is episode 4 is -20.0 Length is episode 4 is 3675
ALE/Pong-v5 (H): Reward is episode 5 is -17.0 Length is episode 5 is 5318
ALE/Pong-v5 (H): Reward is episode 6 is -21.0 Length is episode 6 is 3280
ALE/Pong-v5 (H): Reward is episode 7 is -21.0 Length is episode 7 is 3896
ALE/Pong-v5 (H): Reward is episode 8 is -21.0 Length is episode 8 is 4606
ALE/Pong-v5 (H): Reward is episode 9 is -17.0 Length is episode 9 is 5570
ALE/Pong-v5 (H): Reward is episode 10 is -21.0 Length is episode 10 is 3766
ALE/Pong-v5 (H): Reward is episode 11 is -21.0 Length is episode 11 is 3304
ALE/Pong-v5 (H): Reward is episode 12 is -21.0 Length is episode 12 is 3900
ALE/Pong-v5 (H): Reward is episo